In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install torchsummary torchgeometry segmentation-models-pytorch tqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=c44f5871045b6e4e01621a6f14aff409f39255df0c06154cee35b253b85a2401
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=65a254e60814b520cb5c7bfcdf4ae2559751b01dd1d1e25c99fc23fe522ab2c6
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c601428

In [27]:
!pip install tqdm


In [4]:
# import libraries
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.transforms import ToTensor
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
from torchinfo import summary
import timm
import segmentation_models_pytorch as smp
import wandb

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [29]:
# configuration
BATCH_SIZE = 8
lr = 0.01
NUM_EPOCHS = 50

In [5]:
# dataset
class DatasetCustom(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize)
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [8]:
cd ..

/


In [11]:
# load images + masks
images_path = "/kaggle/input/polyp-dataset/train/train/"
image_path = []
TRAIN_DIR = '/kaggle/input/polyp-dataset/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
print(len(image_path))

mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/polyp-dataset/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
print(len(mask_path))

1000
1000


In [15]:
# store images + masks
images_data = []
masks_data = []
for x,y in dataset:
    images_data.append(x)
    masks_data.append(y)

In [16]:
len(images_data)

1000

In [13]:
# define train + val transformation
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.VerticalFlip(p=0.4),
    A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

/tmp/ipykernel_30/3580890972.py:5: UserWarning: Argument 'eps' is not valid and will be ignored.
  A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),


In [18]:
# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, index):
        image = self.data[index]
        label = self.targets[index]
        assert image.shape[:2] == label.shape[:2]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label
    
    def __len__(self):
        return len(self.data)

In [22]:
# split train and eval set
train_size = int(0.8 * len(images_data))
val_size = len(images_data) - train_size
train_dataset = CustomDataset(images_data[:train_size], masks_data[:train_size], transform=train_transformation)
val_dataset = CustomDataset(images_data[train_size:], masks_data[train_size:], transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
# model
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

In [28]:
# login to wandb
wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "e8c9ee40b64158f0b2cbede51f3751b64d9368e4",
)
wandb.init(
    project = "PolypSegment"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: an-nd225432 (an-nd225432-hust). Use `wandb login --relogin` to force relogin


In [33]:
optimizer = optim.Adam(model.parameters(), lr=lr)


In [69]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Decays LR every 10 epochs by 0.1


In [70]:
from tqdm import tqdm
import time

NUM_EPOCHS = 30
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
best_val_loss = 999

epoch_bar = tqdm(total=NUM_EPOCHS, desc='Total Progress')

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)

            val_loss += criterion(outputs.float(),labels.long()).item()

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)

    # Step the scheduler
    scheduler.step()
        
    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader)})
epoch_bar.close()
    
    


Total Progress:  20%|██        | 10/50 [06:39<26:37, 39.94s/it]


Epoch [1/30], Loss: 0.0889906291



Total Progress:   7%|▋         | 2/30 [00:50<11:49, 25.33s/it]

Epoch [2/30], Loss: 0.0900403210
Epoch [3/30], Loss: 0.0851921190



Total Progress:  13%|█▎        | 4/30 [01:42<11:05, 25.61s/it]

...


Total Progress:  80%|████████  | 24/30 [10:21<02:35, 25.98s/it]

Epoch [24/30], Loss: 0.1076800600



Total Progress:  83%|████████▎ | 25/30 [10:47<02:09, 25.98s/it]

Epoch [25/30], Loss: 0.1051756456


KeyboardInterrupt: 

In [73]:
torch.save(checkpoint, f'./kaggle/working/model.pth')

In [37]:
!mkdir prediction


In [39]:
cd ..

/


In [41]:
# mask to rgb
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)   

In [56]:
# transforms for evaluation
from albumentations import Compose, Resize, Normalize

transforms = Compose([
    Resize(512, 512),  # Resize images to 512x512
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalize images
    ToTensorV2()  # Convert to PyTorch tensors, (C, H, W)
])

In [71]:
model.eval()
output_dir = "./kaggle/working/prediction"

for i in os.listdir("/kaggle/input/polyp-dataset/test/test"):
    img_path = os.path.join("/kaggle/input/polyp-dataset/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
        # print(output_mask.shape)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    output_path = os.path.join(output_dir, i)
    cv2.imwrite(output_path, mask_rgb)
    print(f"Saved: {output_path}")

Saved: ./kaggle/working/prediction/f62f215f0da4ad3a7ab8df9da7386835.jpeg
Saved: ./kaggle/working/prediction/aeeb2b535797395305af926a6f23c5d6.jpeg
Saved: ./kaggle/working/prediction/2ed9fbb63b28163a745959c03983064a.jpeg
Saved: ./kaggle/working/prediction/3c84417fda8019410b1fcf0625f608b4.jpeg
Saved: ./kaggle/working/prediction/8fa8625605da2023387fd56c04414eaa.jpeg
Saved: ./kaggle/working/prediction/cb1b387133b51209db6dcdda5cc8a788.jpeg
Saved: ./kaggle/working/prediction/a3657e4314fe384eb2ba3adfda6c1899.jpeg
Saved: ./kaggle/working/prediction/c695325ded465efde988dfb96d081533.jpeg
Saved: ./kaggle/working/prediction/0fca6a4248a41e8db8b4ed633b456aaa.jpeg
Saved: ./kaggle/working/prediction/6f4d4987ea3b4bae5672a230194c5a08.jpeg
Saved: ./kaggle/working/prediction/68d4b4ef4d95ceea11957998906d3694.jpeg
Saved: ./kaggle/working/prediction/559c7e610b1531871f2fd85a04faeeb2.jpeg
Saved: ./kaggle/working/prediction/e5e8f14e1e0ae936de314f2d95e6c487.jpeg
Saved: ./kaggle/working/prediction/3c3ca4d5060a633a

In [72]:
# output the result
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r

MASK_DIR_PATH = '/kaggle/working/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'/kaggle/working/output.csv', index=False)

/kaggle/working/prediction/eff05dec1eb3a70b145a7d8d3b6c0ed7.jpeg
/kaggle/working/prediction/cbb2a365b5574868eb60861ee1ff0b8a.jpeg
/kaggle/working/prediction/4ca6160127cd1d5ff99c267599fc487b.jpeg
/kaggle/working/prediction/db5eb2a0e4b50889d874c68c030b9afe.jpeg
/kaggle/working/prediction/2a365b5574868eb60861ee1ff0b8a4f6.jpeg
/kaggle/working/prediction/633a8d5b2b2b55157b7781e2c706c75c.jpeg
/kaggle/working/prediction/7cb2eb1ef57af2ed9fbb63b28163a745.jpeg
/kaggle/working/prediction/780fd497e1c0e9082ea2c193ac8d551c.jpeg
/kaggle/working/prediction/fdbc3bbc04a3afe1f119f21b248d152b.jpeg
/kaggle/working/prediction/66e057db382b8564872a27301a654864.jpeg
/kaggle/working/prediction/0af3feff05dec1eb3a70b145a7d8d3b6.jpeg
/kaggle/working/prediction/425b976973f13dd311a65d2b46d0a608.jpeg
/kaggle/working/prediction/cf464aa36bf7c09a3bb0e5ca159410b9.jpeg
/kaggle/working/prediction/c4be73749a0d21db70dd094a7f32574d.jpeg
/kaggle/working/prediction/fb905b78a91391adc0bb223c4eaf3372.jpeg
/kaggle/working/predictio

In [49]:
pwd

'/'